In [1]:
# packages
import pandas as pd
import numpy as np

from claspy.segmentation import BinaryClaSPSegmentation

import ruptures as rpt

import stumpy
from aeon.segmentation import find_dominant_window_sizes

from aeon.segmentation import GreedyGaussianSegmenter

from aeon.segmentation import InformationGainSegmenter

from aeon.anomaly_detection import STRAY

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, make_scorer,mean_squared_error
from ruptures.metrics import precision_recall
import matplotlib.pyplot as plt
#from aeon.visualisation import plot_series_with_change_points, plot_series_with_profiles
import seaborn as sns

sns.set_theme()
sns.set_color_codes()

from claspy.tests.evaluation import f_measure,covering

C:\Users\loluc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\aeon\base\__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(


In [2]:
def delnear(arr,range):
    i = 0
    while i < len(arr) - 1:
        # Iniziamo con il primo elemento di un potenziale gruppo
        gruppo_inizio = i
        gruppo_fine = i

        # Cerca gli elementi che fanno parte dello stesso gruppo
        while gruppo_fine < len(arr) - 1 and arr[gruppo_fine + 1] - arr[gruppo_fine] < range:
            gruppo_fine += 1

        # Se esiste un gruppo di più elementi
        if gruppo_fine > gruppo_inizio:
            # Se la distanza tra l'inizio e la fine è minore di 50, elimina l'elemento maggiore (gruppo_fine)
            if arr[gruppo_fine] - arr[gruppo_inizio] < range:
                arr = np.delete(arr, gruppo_fine)
            
            # Elimina tutti gli elementi interni al gruppo
            arr = np.concatenate((arr[:gruppo_inizio + 1], arr[gruppo_fine:]))

        # Procedi con il prossimo gruppo
        i = gruppo_inizio + 1

    return arr

In [3]:
def f1scoremargin(ground_truth, predictions, tolerance):
    """
    Calcola l'F1 score con una finestra di tolleranza sui change points.
    
    :param ground_truth: Lista o array di change points reali
    :param predictions: Lista o array di change points predetti
    :param tolerance: La tolleranza temporale (numero di unità temporali)
    :return: precision, recall, f1-score
    """
    ground_truth = np.array(ground_truth)
    predictions = np.array(predictions)
    
    # Vettori per tracciare quali punti sono stati già associati
    matched_ground_truth = np.zeros(len(ground_truth), dtype=bool)
    matched_predictions = np.zeros(len(predictions), dtype=bool)

    mgt={key: False for key in ground_truth}
    mcp={key: False for key in predictions}
    #print(f'gt:{len(ground_truth)} - cp:{len(predictions)}')
    # True Positives (TP)
    tp = 0
    for i, gt_point in enumerate(ground_truth):
        for j, pred_point in enumerate(predictions):
            if not matched_predictions[j] and abs(gt_point - pred_point) <= tolerance:
                tp += 1
                matched_ground_truth[i] = True
                matched_predictions[j] = True

                mgt[gt_point] = True
                mcp[pred_point] = True
                break
            
    
    # False Positives (FP) - predizioni non corrispondenti a nessun ground truth entro la tolleranza
    fp = np.sum(~matched_predictions)
    
    # False Negatives (FN) - punti del ground truth non corrispondenti a nessuna predizione entro la tolleranza
    fn = np.sum(~matched_ground_truth)
    #print(f'tp:{tp} - fp:{fp} - fn:{fn}')
    #print(mgt)
    #print(mcp)
    # Calcolo di precision, recall e F1-score
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

In [4]:
# Questi sono tutte le features estratte
# kineticglobal,kineticchest,directnesshead,density,leftwristke,rightwristke,leftankleke,rightankleke,headke,posturaltension

# Queste sono le feature che utilizzeremo
# kineticglobal, density, leftwirstke, rightwristke, leftankleke, rightankleke, headke, posturaltension
# che corrispondono
# 2,4,5,6,7,8,9,10,11
# Questa funzione legge il file di input e restituisce un dataframe con i dati preprocessati
def ReadAndPreProcess(inputDataRaw):
    # lettura
    df=pd.read_csv(inputDataRaw,sep=' ', header=None)
    df.drop(0, axis=1)
    df["som"]=df.sum(axis=1)
    ## preprocessing

    # dealing NaN values
    #-serafino ha usato forward fill, backward fill, linear interpolation
    #-ricordo che serafino aveva gia utilizzato sta cosa sui dati grezzi non sulle feature ma sui dati prefeature percio dovrebbe essere gia apposto

    # downsampling
    #-sono a 100ms, non sò se devo scendere a 50ms. da decidere
    #-non lo faccio xk non mi interessa se va piu lento, guarda su notion per ulteriori informazioni

    # low pass filter
    #-Skogstad and colleagues (2013) e https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units
    #-implementare dopo

    # remove outliers
    #-utilizzare hampel filter

    # stretch
    #-forse devo stretcharlo come ho fatto precedentemente
    #-anche se nel codice precedente ho stretchato solo il groundtruth

    # ritorno un oggetto dataframe dopo che è stato lavorato, ottenendo un prodotto lavorato
    return df


In [5]:
# questa funzione ritorna un dataframe del groundtruth che viene usato specificatamente per visualizzare il gt
# è soggetto a un preprocessing dei dati siccome l'ultimo groundtruth è dove termina il ts del gt
# di conseguenza per farlo corrispondere, bisogna stretcharlo
# ma ricordo di aver rifatti i dati nuovi per generare un groundtruth a fine ts, da controllare cosi che non serve stretcharlo?
def LoadingGroundTruth(df,gtraw):
    gt=pd.read_csv(gtraw,sep=' ', header=None)
    gt=gt.iloc[:,0].values
    #stretching dei dati se necessario per farlo corrispondere alla ts dei dati
    stretch_gt = np.array([])
    for idx,i in enumerate(gt):
        relpos = len(df)*i/gt[-1]
        stretch_gt = np.append(stretch_gt,relpos)

    # eliminiamo l'ultimo elemento che è stato annotato solo per delimitare la lunghezza della gt simile alla ts
    return stretch_gt[:-1]

In [6]:
# utilizzo CLASP 
# prende come parametro un dataframe e restituisce il clasp score
# gt e known vengono usati per usare il numero vero di cp se uguale a 1 sennò si cerca di predirlo se il modello lo permette
def GetClasp(df,gt,known, **kwargs):
    
    result=np.array([])
    eachresult = []
    eachclasp=[]
    for idx,i in enumerate([2,4,5,6,7,8,9,10,11]):
      
        ts=df.iloc[:,i]
        clasp = BinaryClaSPSegmentation(**kwargs)
        found_cps = clasp.fit_predict(ts.values)    

        # c'è un bug con binseg dove un cp è oltre la lunghezza del ts
        # faccio un loop e se eccede cambio il valore con la len(tf)-1
        for i in range(0,len(found_cps)):
            if found_cps[i] >= len(ts):
                found_cps[i] = len(ts)-1

        # per ogni array di cp di ogni singola feature
        # li unisco in un unico array. in pratica faccio un OR di tutti i cp
        result = np.sort(np.append(result,found_cps).flatten())
        result = np.unique(result)
        eachresult.append(found_cps)
        eachclasp.append(clasp)
        
        

        
    return result, eachresult, eachclasp
        
    


In [7]:
# calcola i vari scores dati il groundtruth e il prediction
# puo salvare il risultato su file per evitare di perderli
# prende come parametro nome del groundtruth, groundtruth, nome della timeseries e il prediction
def Evaluate(modelName,gtName, gt, tsName, cp, df, margin,msg, nomeFile):
    # creo dei array di lunghezza come la ts cosi possono fare il confronto
    # sia per il gt che per il pd
  
    cpnump = np.array(cp)
    gtnump = np.array(gt)

    cp_long = np.zeros(len(df)+1)
    cp_long[cpnump.astype(int)]=1

    gt_long = np.zeros(len(df)+1)
    gt_long[gtnump.astype(int)]=1

    # calcolo lo score 
    print(f'f1margin: {f1scoremargin(gt.astype(int),cp.astype(int),margin)}')
    precision,recall,f1=f1scoremargin(gt.astype(int),cp.astype(int),margin)

    return precision,recall,f1
     #scrivo su file il risultato

    """
    f = open("outputFile/ClaSPallresult", "a")
    f.write(nomeFile+"\n")
    f.write(msg+"\n")
    f.write("precision:"+str(precision)+" recall:"+str(recall)+" f1:"+str(f1)+" \n")
    f.write("\n")
    f.close()
    """
    

In [8]:
def Plotclasp(eachclasp,gt,margin,eachcp,nomeFile):
    
    for idx,clasp in enumerate(eachclasp):
        clasp.plot(gt_cps=gt.astype(int), heading=f'f1margin: {f1scoremargin(gt.astype(int),eachcp[idx].astype(int),margin)} {nomeFile}', ts_name="suss", file_path="segmentation_example.png")


    
        for idx2,j in enumerate(gt.astype(int)):
            plt.fill_betweenx(np.array([0, 1]), j-margin, j+margin, color='green', alpha=0.3)


In [9]:
def PlotResult(df,gt,cp, nomeFile, margin):
    #da testare quando ho piu valori
    #clasp.plot(gt_cps=gt.astype(int), heading="Segmentation of different umpire cricket signals", ts_name="ACC", file_path="segmentation_example.png")

    plt.figure(figsize=(18,9))
    plt.plot(np.arange(len(df["som"].values)),df["som"].values,'blue',linewidth=0.5)
    for idx2,i in enumerate(gt.astype(int)):

            plt.axvline(x = i, color = 'green',linewidth=1) 
            
    for j in cp.tolist():
        plt.axvline(x = j, color = 'red',linewidth=1,linestyle="-.") 

    for idx2,k in enumerate(gt.astype(int)):
            plt.fill_betweenx(np.array([0, 1]), k-margin, k+margin, color='green', alpha=0.3)
    plt.xlabel(f'{nomeFile} {f1scoremargin(gt.astype(int),cp.astype(int),margin)}')



In [10]:
timeseries=[
    "in\cora1_input.txt",
      "in\cora4_input.txt",
      "in\cora5_input.txt",
      "in\cora14_input.txt",
      "in\marianne7_input.txt",
      "in\marianne8_input.txt",
      "in\marianne10_input.txt",
      "in\marianne18_input.txt",
      "in\marianne19_input.txt",
      "in\marianne24_input.txt",
      "in\marianne26_input.txt",
      "in\marianne41_input.txt",
      "in\marianne42_input.txt",
      "in\marianne43_input.txt",
      "in\marianne47_input.txt",
      "in\marianne48_input.txt",
      "in\muriel18_input.txt",
      "in\muriel26_input.txt",
      "in\muriel27_input.txt",
      "in\muriel30_input.txt"

      ]
groundtruth=[
         "gt\cora_gt_2019-08-08_t001_video01.txt",
         "gt\cora_gt_2019-08-08_t004_video01.txt",
         "gt\cora5_gt.txt",
         "gt\cora_gt_2019-08-08_t014_video01.txt",
         "gt\marianne_gt_2016-03-22_t007_video01.txt",
         "gt\marianne_gt_2016-03-22_t008_video01.txt",
         "gt\marianne_gt_2016-03-22_t010_video01.txt",
         "gt\marianne_gt_2016-03-22_t018_video01.txt",
         "gt\marianne_gt_2016-03-22_t019_video01.txt",
         "gt\marianne_gt_2016-03-22_t024_video01.txt",
         "gt\marianne_gt_2016-03-22_t026_video01.txt",
         "gt\marianne_gt_2016-03-22_t041_video01.txt",
         "gt\marianne_gt_2016-03-22_t042_video01.txt",
         "gt\marianne_gt_2016-03-22_t043_video01.txt",
         "gt\marianne_gt_2016-03-22_t047_video01.txt",
         "gt\marianne_gt_2016-03-22_t048_video01.txt",
         "gt\muriel_gt_2016-03-21_t018_video01.txt",
         "gt\muriel_gt_2016-03-21_t026_video01.txt",
         "gt\muriel_gt_2016-03-21_t027_video01.txt",
         "gt\muriel_gt_2016-03-23_t030_video01.txt"
         ]


In [11]:
cleaned_array = [s[3:-10] for s in timeseries]
outdf = pd.DataFrame({"name":cleaned_array})



outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=5,score="roc_auc")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["5_rocauc"] = outeval



outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=5,score="f1")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["5_f1"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=10,score="roc_auc")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["10_rocauc"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=10,score="f1")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["10_f1"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=25,score="roc_auc")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["25_rocauc"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=25,score="f1")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["25_f1"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=50,score="roc_auc")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["50_rocauc"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=50,score="f1")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["50_f1"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=75,score="roc_auc")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["75_rocauc"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=75,score="f1")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["75_f1"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=100,score="roc_auc")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["100_rocauc"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=100,score="f1")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["100_f1"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=150,score="roc_auc")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["150_rocauc"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=150,score="f1")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["150_f1"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=200,score="roc_auc")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["200_rocauc"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=200,score="f1")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["200_f1"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=300,score="roc_auc")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["300_rocauc"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=300,score="f1")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["300_f1"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=500,score="roc_auc")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["500_rocauc"] = outeval

outeval=[]
for i in range(len(timeseries)):
    print(i)
    df=ReadAndPreProcess(timeseries[i])
    gt=LoadingGroundTruth(df,groundtruth[i])
    cp,eachcp,clasp=GetClasp(df,gt,0,distance="euclidean_distance",excl_radius=4,window_size=500,score="f1")
    #PlotResult(df,gt,cp, timeseries[i], 100)
    cp=delnear(cp,100)
    precision,recall,f1=Evaluate("CLASP",groundtruth[i],gt,timeseries[i],cp,df,100,"suss",timeseries[i])
    outeval.append(f1)
outdf["500_f1"] = outeval

outdf.to_excel("outputFile/outputWINDOWCHECK.xlsx")

0


C:\Users\loluc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\claspy\nearest_neighbour.py:252: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  start, end = pranges[idx]


f1margin: (0.5, 0.28125, 0.36)
1
f1margin: (0.6666666666666666, 0.25, 0.36363636363636365)
2
f1margin: (0.3888888888888889, 0.8235294117647058, 0.5283018867924528)
3
f1margin: (1.0, 0.04, 0.07692307692307693)
4


C:\Users\loluc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\stats\_stats_py.py:9205: RuntimeWarning: invalid value encountered in scalar divide
  z = (s - expected) / np.sqrt(n1*n2*(n1+n2+1)/12.0)


f1margin: (0.0, 0.0, 0)
5
f1margin: (1.0, 0.07407407407407407, 0.13793103448275862)
6
f1margin: (1.0, 0.07692307692307693, 0.14285714285714288)
7
f1margin: (0.5, 0.038461538461538464, 0.07142857142857144)
8
f1margin: (1.0, 0.06666666666666667, 0.125)
9
f1margin: (1.0, 0.045454545454545456, 0.08695652173913045)
10
f1margin: (0, 0.0, 0)
11
f1margin: (0.5, 0.07692307692307693, 0.13333333333333336)
12
f1margin: (0.0, 0.0, 0)
13
f1margin: (0.3333333333333333, 0.1282051282051282, 0.18518518518518517)
14
f1margin: (0, 0.0, 0)
15
f1margin: (1.0, 0.058823529411764705, 0.1111111111111111)
16
f1margin: (1.0, 0.08571428571428572, 0.15789473684210528)
17
f1margin: (0.0, 0.0, 0)
18
f1margin: (1.0, 0.012658227848101266, 0.024999999999999998)
19
f1margin: (0.75, 0.15789473684210525, 0.2608695652173913)
0
f1margin: (0.3333333333333333, 0.09375, 0.14634146341463417)
1
f1margin: (1.0, 0.25, 0.4)
2
f1margin: (0.45161290322580644, 0.8235294117647058, 0.5833333333333333)
3
f1margin: (1.0, 0.16, 0.2758620689

...

f1margin: (0.09375, 0.3333333333333333, 0.14634146341463417)
13
f1margin: (0.45, 0.6923076923076923, 0.5454545454545455)
14


C:\Users\loluc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\claspy\segmentation.py:212: UserWarning: Time series must at least have 2*min_seg_size data points for segmentation. Try setting a smaller window size.
  warnings.warn(


f1margin: (1.0, 0.29411764705882354, 0.45454545454545453)
15


C:\Users\loluc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\claspy\segmentation.py:212: UserWarning: Time series must at least have 2*min_seg_size data points for segmentation. Try setting a smaller window size.
  warnings.warn(


f1margin: (0.7, 0.4117647058823529, 0.5185185185185185)
16
f1margin: (0.7222222222222222, 0.37142857142857144, 0.490566037735849)
17
f1margin: (0.5625, 0.24, 0.33644859813084116)
18
f1margin: (0.8666666666666667, 0.16455696202531644, 0.2765957446808511)
19
f1margin: (0.5641025641025641, 0.5789473684210527, 0.5714285714285715)
